In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\mtsia\AppData\Local\Temp\ipykernel_17376\896178439.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import numpy as np
import pandas as pd
import random
import scipy
from scipy import sparse
import scipy.sparse.linalg as linalg
from scipy.sparse import identity

In [3]:
#this is a compressed sparse matrix representation 
edges = np.loadtxt("out.web-BerkStan", dtype=int, skiprows=2)
edges[10:20], edges.shape

(array([[ 9, 12],
        [ 9, 13],
        [ 9, 14],
        [ 9, 15],
        [ 9, 16],
        [ 9, 17],
        [ 9, 18],
        [ 9, 19],
        [ 9, 20],
        [ 9, 21]]),
 (7600595, 2))

In [4]:
def create_matrix(matrix):
    
    A, index, inverse_index = np.unique(matrix, return_index = True, return_inverse = True)
    rows, columns = inverse_index.reshape(matrix.shape).T
    B = sparse.csr_matrix((np.ones(rows.shape,int), (rows,columns))) 
    
    #The sparse matrix B only indicates which nodes are connected(using 1's). We need to convert these into transitional probabilities.
    #firstly, get sum of every row
    sum_of_rows = np.asarray(B.sum(axis=1)).reshape(-1)
    #keep only non-zero sums
    nonzero = sum_of_rows.nonzero()[0]
    
    #Dangling Nodes (nodes with no outlinks): all rows containing 0 must be replaced
    #We create a vector a where a_i = 1 if the i-th row of corresponds to a dangling node, and 0 otherwise
    a = np.asarray([1 if sum_of_rows[i] == 0 else 0 for i in range(len(sum_of_rows))])
    
    # we store the value for each row in its diagonal spot to avoid creating a dense matrix
    probability_matrix = sparse.csr_matrix((1 / sum_of_rows[nonzero], (nonzero, nonzero)), shape=(B.shape[0], B.shape[0]))
    P = probability_matrix @ B
    
    return P, a

In [5]:
P, a = create_matrix(edges)

In [6]:
P.shape
x_old = np.zeros(P.shape[0])
v = np.ones(P.shape[0]) / P.shape[0]
x_old.shape, P.shape, v.shape, a.shape

((685230,), (685230, 685230), (685230,), (685230,))

## (1)

### Power Method 

In [7]:
def power_method(probability_matrix, a, alpha, tol=1e-08, converge=False): 
    
    
    v = np.ones(probability_matrix.shape[0]) / probability_matrix.shape[0]
    
    #initial vector: equal probability of going to all nodes
    x_old = np.ones(probability_matrix.shape[0])
    x_new = (np.ones(probability_matrix.shape[0]) / probability_matrix.shape[0])
    differences = []
    
    iterations = 0
    while True:
        x_old = x_new 
        x_new = alpha * (probability_matrix @ x_old) + v.dot(a @ x_old) + (1-alpha)*v
        differences.append((abs(np.subtract(x_new, x_old))))
        iterations += 1
        
        if scipy.linalg.norm(x_new - x_old) < tol:
            break
         
    if converge == True:
        print(f'Number of Iterations: {iterations}')
        return x_new, np.array(differences)
    
    else:
        print(f'Number of Iterations: {iterations}')
        return x_new

In [8]:
%%time
power = power_method(P,a, 0.85, converge=False)
power.shape, power[:10]

Number of Iterations: 37
CPU times: total: 1.36 s
Wall time: 1.3 s


((685230,),
 array([1.44929960e-06, 1.44902453e-06, 1.44897607e-06, 1.45267094e-06,
        1.44588451e-06, 1.44901529e-06, 1.45080008e-06, 1.45146103e-06,
        1.43963615e-06, 1.42398979e-06]))

### Linear Method 

In [9]:
def linear_system(probability_matrix, a, alpha):
    
    v = np.ones(probability_matrix.shape[0]) / probability_matrix.shape[0]
    v1 = v[a==0] #positions of nondangling nodes
    P1 = probability_matrix[:, (a==0)][(a==0), :]
    ident = sparse.eye(P.shape[0], format='csr')[:, (a==0)][(a==0), :]
    v2 = v[a==1] #positions of dangling nodes
    P2 = probability_matrix[:, (a==1)][(a==0), :] #rows nondangling, columns dangling
    
    A = scipy.sparse.csc_matrix(ident - alpha * P1)
    b = scipy.sparse.csc_matrix(v1).T
    pi_1 = linalg.spsolve(A, b) #nondangling nodes
    c = scipy.sparse.csc_matrix(pi_1)
    pi_2 = alpha * (c @ P2) + v2 #dangling nodes
    p = sparse.hstack([pi_1, pi_2]).toarray()
    
    return (p / np.linalg.norm(p, 1)).T.reshape(-1)

In [10]:
%%time
linear = linear_system(P,a, 0.85)
linear[:10]

CPU times: total: 2min 43s
Wall time: 2min 1s


array([0.0279987 , 0.02799243, 0.02799132, 0.02807545, 0.02792096,
       0.02799221, 0.02803285, 0.02804789, 0.02777872, 0.02742207])

In [11]:
ordered_power = np.argsort(power)
ordered_linear = np.argsort(linear)
ordered_power[::-1][:100], ordered_linear[::-1][:100]

(array([139321, 437047, 437074, 582062, 142166, 142220, 142221, 142222,
        142223, 584999, 425992, 590213, 126309, 584955, 584982, 121020,
        460203, 584909, 584973, 585006, 200498, 200501, 585018, 584940,
        585014, 584930, 584931, 585015, 183771, 183709, 585017, 584939,
        183773, 584996, 584938, 584933, 183779, 584934, 584997, 584927,
        584935, 585020, 584937, 585024, 584924, 584926, 183733, 584915,
        183753, 183752, 584910, 183746, 183744, 183731, 584918, 183728,
        183726, 183671, 183693, 183700, 183706, 585042, 584919, 585026,
        584923, 584925, 183760, 585012, 585027, 575912, 585028, 183642,
        585039, 585032, 183644, 585035, 585036, 584920, 585038, 585013,
        584971, 585011, 584960, 584946, 584985, 584984, 584948, 584949,
        584950, 584951, 584983, 585010, 584956, 584961, 584945, 584962,
        584963, 584965, 584966, 584977], dtype=int64),
 array([682507, 684215, 682054, 685207, 680980, 684573, 680536, 680529,
        6

The two methods give different pagerank vectors, as the power method only takes into account the absolute largest eigenvalue of the matrix P. The power method is faster, as it only takes 1.63 s whereas the linear system method takes 2min and 10s.

## (2)

In [12]:
%%time
power = power_method(P, a, 0.99)

Number of Iterations: 357
CPU times: total: 14.3 s
Wall time: 12.8 s


In [13]:
%%time
linear = linear_system(P,a, 0.99)

CPU times: total: 2min 56s
Wall time: 2min 13s


In [14]:
ordered_power_2 = np.argsort(power)
ordered_linear_2 = np.argsort(linear)
ordered_power_2[::-1][:100], ordered_linear_2[::-1][:100]

(array([122712, 123357, 120369, 122710, 123355, 120371, 122951, 122778,
        122761, 122807, 122912, 120448, 122958, 122895, 120549, 120541,
        122863, 122773, 122759, 120500, 122871, 122781, 120524, 122846,
        122789, 122837, 122834, 122822, 122760, 122917, 122707, 122749,
        122742, 122743, 120590, 120415, 120368, 120330, 122740, 120414,
        122952, 122655, 120573, 122953, 122709, 122840, 120485, 120484,
        120433, 123505, 122833, 122832, 122820, 125127, 120595, 120533,
        122945, 122870, 122944, 120434, 120534, 123543, 123498, 122894,
        122899, 123486, 122928, 123528, 122808, 120365, 122708, 122788,
        122779, 122793, 122785, 122750, 122747, 122769, 122925, 122923,
        120547, 122869, 122874, 122922, 122755, 122756, 122775, 120499,
        120501, 122864, 122926, 120574, 122865, 122754, 122782, 122948,
        120532, 122927, 120578, 122753], dtype=int64),
 array([682054, 684215, 685207, 682507, 680536, 680980, 683467, 680951,
        6

We now give our algorithm bigger tolerance in ordering the nodes. The power method vector has significantly changed, whereas the linear system vector is more stable.

## (3) 

In [15]:
%%time
power, differences = power_method(P,a, 0.85, converge=True)

Number of Iterations: 37
CPU times: total: 1.56 s
Wall time: 1.37 s


In [16]:
iterations_needed = np.where(differences >= 1e-08, 0, 1).sum(axis = 0)
pd.Series(iterations_needed).value_counts().sort_values()

23         2
24         2
25         4
26         6
27        61
29       662
28       845
30      1138
31      1922
32      4239
34      7745
33     12445
35     13549
36    642610
dtype: int64

In [17]:
print(f'These are the 50 nodes that converge the fastest: {np.argsort(iterations_needed)[:50]}')
print(f'Distance of the 500 probabilities of the fastest nodes from the maximum pagerank probability(of the strongest node): {(np.linalg.norm(power[np.argsort(iterations_needed)[:500]] - max(power),2)):.20f}')
print(f'Distance of the 500 probabilities of the fastest nodes from the minimum pagerank probability(of the weakest node): {(np.linalg.norm(power[np.argsort(iterations_needed)[:500]] - min(power),2)):.20f}')

These are the 50 nodes that converge the fastest: [118745 135259 118766 434971 434909 434981 600537 436691 434970 600535
 434986 118764 434972 436690 466357 600626 466342 600628 496840 600629
  55812 600627 403219 453973 453972 606611 466290 451690 466298 518640
 466299 466349 496841 518641 466348 466303 466304 466305 466347 466289
 466345 199663 466288 403225 466286 466350 453919 453920 453921 453922]
Distance of the 500 probabilities of the fastest nodes from the maximum pagerank probability(of the strongest node): 0.00000945746977215995
Distance of the 500 probabilities of the fastest nodes from the minimum pagerank probability(of the weakest node): 0.00001939718957613864


Most of these nodes seem to have small probabilities (they are very close to the smallest probability in the pagerank vector), so we assume that the weakest nodes are the ones that converge the fastest.

## (4)

In [18]:
power[np.argsort(power)[::-1][:10]] #Max to min probbility

array([1.46953795e-06, 1.46953795e-06, 1.46953795e-06, 1.46953795e-06,
       1.46953795e-06, 1.46953795e-06, 1.46953795e-06, 1.46953795e-06,
       1.46953795e-06, 1.46953795e-06])

In [19]:
#Add extra node outlinks 10 most important ones and inlinks 500 below average ones
print(f'The new node is {edges.max() + 1}')
np.random.seed(123)

outlinks = [] #add information for outlinks
for i in range(0,10):
    outlinks.append((edges.max() + 1, np.argsort(power)[::-1][i]))

inlinks = [] #add information for inlinks
weakest = np.array_split(np.argsort(power)[::-1] ,2)[1]
weakest_500 = np.random.choice(weakest, 500, replace=False)
for i in range(0,500):
    inlinks.append((weakest_500[i] ,edges.max() + 1))

new_edges = np.vstack([edges, outlinks, inlinks])

The new node is 685231


In [20]:
P, a = create_matrix(new_edges)
power = power_method(P, a, 0.85, converge=False)
power.shape, power[:10]

Number of Iterations: 37


((685231,),
 array([1.44930846e-06, 1.44903384e-06, 1.44898334e-06, 1.45267681e-06,
        1.44588914e-06, 1.44901936e-06, 1.45083534e-06, 1.45146289e-06,
        1.43971158e-06, 1.42397465e-06]))

In [21]:
print(f'Position of the new node: {np.where(np.argsort(power)[::-1] == np.argsort(power).max())[0][0]} out of {np.argsort(power).shape[0]} nodes, somewhat in the middle.')

Position of the new node: 388178 out of 685231 nodes, somewhat in the middle.


## (5)

In [22]:
#Add extra node outlinks 500 below average ones and inlinks 10 most important ones
print(f'The new node is {edges.max() + 1}')
np.random.seed(123)

outlinks = [] #add information for inlinks
weakest = np.array_split(np.argsort(power)[::-1] ,2)[1]
weakest_500 = np.random.choice(weakest, 10, replace=False)
for i in range(0,10):
    outlinks.append((edges.max() + 1, weakest_500[i]))
    
inlinks = [] #add information for inlinks
for i in range(0,10):
    inlinks.append((np.argsort(power)[::-1][i], edges.max() + 1,))

new_edges = np.vstack([edges, outlinks, inlinks])

The new node is 685231


In [23]:
P, a = create_matrix(new_edges)
power = power_method(P, a, 0.85, converge=False)
power.shape, power[:10]

Number of Iterations: 37


((685231,),
 array([1.44929747e-06, 1.44902240e-06, 1.44897394e-06, 1.45266881e-06,
        1.44588238e-06, 1.44901316e-06, 1.45079795e-06, 1.45145890e-06,
        1.43963404e-06, 1.42398770e-06]))

In [24]:
print(f'Position of the new node: {np.where(np.argsort(power)[::-1] == np.argsort(power).max())[0][0]} out of {np.argsort(power).shape[0]} nodes. The importance of the new node dropped significantly.')

Position of the new node: 550143 out of 685231 nodes. The importance of the new node dropped significantly.


In [25]:
np.argsort(power)[::-1][:10], ordered_power[::-1][:10] 

(array([111551, 111634, 111619, 111684, 111769, 111770, 111771, 111616,
        111772, 111773], dtype=int64),
 array([139321, 437047, 437074, 582062, 142166, 142220, 142221, 142222,
        142223, 584999], dtype=int64))

The first 10 most important nodes have also changed -- this makes sense since the new node that is inlinked from important nodes now outlinks to the 10 most non-important ones, deeming them isnignificant.